In [1]:
import numpy as np # type: ignore
from sklearn.linear_model import LinearRegression
from scipy.stats import t
import random
import pandas as pd

In [2]:
def simulate_genotypes(num_individuals, freq_ref):
    genotypes=[]
    for i in range(num_individuals):
        p_AA = freq_ref ** 2
        p_Aa = 2 * freq_ref * (1 - freq_ref)
        p_aa = (1 - freq_ref) ** 2
            
            # Use random.choices() with weighted probabilities
        
        gen = random.choices([0, 1, 2], weights=[p_AA, p_Aa, p_aa])[0]
        genotypes.append(gen)
    return genotypes

In [3]:
# Parameters
n_samples = 500  # Number of samples per dataset
n_iterations = 500  # Number of simulations
beta3_values =[ 0,0.2 ]
'''[0, 0.25, 0.5, 0.75, 1]'''
power=pd.DataFrame()
power['beta3']= beta3_values
power_results=[]
interaction_results = { 'genotype_error': [], 'exposure_error': [], 'both_error': [],'no_error': []}


# Simulation loop
cases=[ 'genotype_error', 'exposure_error', 'both_error','no_error']
for beta3 in beta3_values:
    print(beta3)
    for case in cases:
        for _ in range(n_iterations):
            # Step 1: Simulate Genotype (G) and Exposure (E)
            G = simulate_genotypes(n_samples,0.6)
            data1=pd.DataFrame({'genotype':G})
            E = np.random.binomial(1,0.6,n_samples)
            data1['Exposure']=E
            # Step 2: Calculate Phenotype (Y) with interaction and noise
            noise = np.random.normal(0, 1, n_samples)

            beta_gen = 0.3
            beta_exp = 0.5
            # interaction_effect = 0.2
            data1['phenotype']= (beta_gen * data1['genotype'] +
                            beta_exp* data1['Exposure'] +
                            beta3 * data1['genotype'] * data1['Exposure'] +
                            np.random.normal(0, 1, n_samples))
            #errors
            # adding error to genotype 
            rows_for_gen = np.random.choice(data1.index, 10, replace=False)
            data1['genotype_with_error']=data1['genotype']
            for row in rows_for_gen:
                original_genotype = data1.loc[row, 'genotype']
            
                possible_values = [0, 1, 2]
                possible_values.remove(original_genotype)
            
                new_genotype = np.random.choice(possible_values)
                data1.loc[row, 'genotype_with_error'] = new_genotype

            # adding error to exposure 
            rows = np.random.choice(data1.index, 20, replace=False)
            data1['Exposure_with_error']=data1['Exposure']
            for row in rows:
                data1.loc[row,'Exposure_with_error']= 1-data1.loc[row,'Exposure']


            data1['interaction_without_error']=data1['genotype'] * data1['Exposure']
            data1['interaction_with_gen_error']=data1['genotype_with_error']*data1['Exposure']
            data1['interaction_with_exp_error']=data1['Exposure_with_error']*data1['genotype']
            data1['interaction_with_both_error']=data1['genotype_with_error']*data1['Exposure_with_error']
            
            x_1=data1[['genotype_with_error','Exposure','interaction_with_gen_error']]
            x_2=data1[['genotype','Exposure_with_error','interaction_with_exp_error']]
            x_3=data1[['genotype_with_error','Exposure_with_error','interaction_with_both_error']]
            x_4=data1[['genotype','Exposure','interaction_without_error']]
            y=data1['phenotype']


            # Step 3: Design matrix with interaction term and intercept
            if case=='genotype_error':
                X = np.column_stack((np.ones(n_samples), data1['genotype_with_error'], data1['Exposure'], data1['interaction_with_gen_error']))
            elif case=='exposure_error':
                X = np.column_stack((np.ones(n_samples), data1['genotype'], data1['Exposure_with_error'], data1['interaction_with_exp_error']))
            elif case=='both_error':
                X = np.column_stack((np.ones(n_samples), data1['genotype_with_error'], data1['Exposure_with_error'], data1['interaction_with_both_error']))
            else:
                X = np.column_stack((np.ones(n_samples), data1['genotype'], data1['Exposure'], data1['interaction_without_error']))

            # Step 4: Fit the regression model
            model = LinearRegression()
            model.fit(X, y)
            
            # Step 5: Calculate residual variance (sigma^2)
            residuals = y - model.predict(X)
            sigma_squared = np.var(residuals, ddof=X.shape[1])
            
            # Step 6: Compute (X^T X)^-1
            X_transpose_X_inv = np.linalg.inv(X.T @ X)
            
            # Step 7: Standard error for interaction coefficient
            interaction_idx = -1  # Index of interaction term
            SE_interaction = np.sqrt(sigma_squared * np.diag(X_transpose_X_inv)[interaction_idx])
            
            # Step 8: t-statistic for interaction coefficient
            interaction_coef = model.coef_[interaction_idx]
            t_stat = interaction_coef / SE_interaction
            
            # Step 9: Degrees of freedom and p-value
            dof = n_samples - X.shape[1]
            p_value = 2 * (1 - t.cdf(np.abs(t_stat), df=dof))
            
            # Store results
            interaction_results[case].append((interaction_coef, p_value,t_stat))

    for case in interaction_results:
        interaction_coeffs, p_values , t_stat= zip(*interaction_results[case])
        p_val=np.mean(p_values)
        mean_coeff = np.mean(interaction_coeffs)
        std_coeff = np.std(interaction_coeffs)
        test_stat = np.mean(t_stat)
        rejection_rate = np.mean(np.array(p_values) < 0.05)
        print(case)
        # print(f"Mean Interaction Coefficient: {mean_coeff:.4f}")
        print(f'Mean test stat:{test_stat}')
        # print(f"Standard Deviation of Interaction Coefficient: {std_coeff:.4f}")
        print(f"Rejection Rate (p < 0.05): {rejection_rate:.2%}")
        print(f"P-value: {p_val}")
    
    power_results.append(rejection_rate)
print(power_results)
# Step 10: Analyze results



0
genotype_error
Mean test stat:-0.043280940400660384
Rejection Rate (p < 0.05): 5.80%
P-value: 0.5301361740791792
exposure_error
Mean test stat:0.055317608184986156
Rejection Rate (p < 0.05): 4.80%
P-value: 0.5042388418080439
both_error
Mean test stat:0.022071746131067615
Rejection Rate (p < 0.05): 5.40%
P-value: 0.5042144284378384
no_error
Mean test stat:0.0051255538057715915
Rejection Rate (p < 0.05): 7.40%
P-value: 0.4876202838497919
0.2
genotype_error
Mean test stat:0.6576849757738271
Rejection Rate (p < 0.05): 17.20%
P-value: 0.40623798089982577
exposure_error
Mean test stat:0.6905293454169166
Rejection Rate (p < 0.05): 16.50%
P-value: 0.3937487551980547
both_error
Mean test stat:0.6740811449348055
Rejection Rate (p < 0.05): 15.70%
P-value: 0.40163738611996197
no_error
Mean test stat:0.766395360206729
Rejection Rate (p < 0.05): 20.40%
P-value: 0.3639778225254793
[0.074, 0.204]
